# Лабораторная работа №5

## Задание 1 
###### Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [9]:
data = pd.read_csv('database.csv',low_memory=False)
# Удаление ненужных столбцов
data = data.drop(columns=["Record ID", "Record Source"])
# Заполнение пропущенных значений
data = data.dropna()

###### Выбираем случайное подмножество данных, т.к. загруженный датасет содержит 638 454 записей которые будут обрабатываться слишком долго

In [10]:
subset_data = data.sample(n=11000, random_state=42)

subset_data.head() # Просмотр первых нескольких строк данных

,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,Crime Solved,...,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,CA03612,Victorville,Municipal Police,San Bernardino,California,2005,July,1,Murder or Manslaughter,No,...,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,TX10100,Harris County,Sheriff,Harris,Texas,2013,May,3,Murder or Manslaughter,No,...,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,AR06200,St. Francis,Sheriff,St. Francis,Arkansas,2001,June,1,Murder or Manslaughter,Yes,...,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,AZ01003,Tucson,Municipal Police,Pima,Arizona,1998,March,6,Murder or Manslaughter,No,...,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,WA02400,Okanogan,Sheriff,Okanogan,Washington,1999,August,1,Murder or Manslaughter,Yes,...,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


###### Выбор признаков и целевой переменной

In [11]:
X = subset_data[['Victim Sex', 'Victim Age', 'Victim Race', 'Perpetrator Sex', 'Perpetrator Age', 'Perpetrator Race', 'Weapon']]
Y = subset_data['Crime Solved']
# можно заменить data на subset_data и наоборот

###### Преобразование категориальных признаков в числовые

In [12]:
X = pd.get_dummies(X)

###### Разделение данных на обучающий и тестовый наборы

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

###### Определение параметров для решающего дерева

In [21]:
dt_param_grid = {
    'max_depth': [3, 5, 7, 10]
}

###### Определение параметров для случайного леса

In [22]:
rf_param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15]
}

###### Создание экземпляров классификаторов

In [23]:
dt_classifier = DecisionTreeClassifier()
rf_classifier = RandomForestClassifier()

###### Создание объектов GridSearchCV

In [26]:
dt_grid_search = GridSearchCV(dt_classifier, dt_param_grid, cv=5)
rf_grid_search = GridSearchCV(rf_classifier, rf_param_grid, cv=5)

###### Обучение моделей с использованием Grid Search

In [28]:
dt_grid_search.fit(X_train, Y_train)
rf_grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [5, 10, 15],
                         'n_estimators': [50, 100, 150]})

###### Вывод результатов кросс-валидации для решающего дерева

In [29]:
print("Decision Tree Cross-Validation Results:")
print(pd.DataFrame(dt_grid_search.cv_results_))

Decision Tree Cross-Validation Results:
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.046124      0.007271         0.009941        0.003722   
1       0.050009      0.005324         0.011646        0.001862   
2       0.051776      0.008841         0.014752        0.006693   
3       0.055335      0.009641         0.010773        0.000730   

  param_max_depth             params  split0_test_score  split1_test_score  \
0               3   {'max_depth': 3}           0.999432                1.0   
1               5   {'max_depth': 5}           0.999432                1.0   
2               7   {'max_depth': 7}           0.999432                1.0   
3              10  {'max_depth': 10}           0.999432                1.0   

   split2_test_score  split3_test_score  split4_test_score  mean_test_score  \
0           0.998295           1.000000                1.0         0.999545   
1           0.998295           0.999432                1.0         0.999432

###### Вывод результатов кросс-валидации для случайного леса

In [30]:
print("\nRandom Forest Cross-Validation Results:")
print(pd.DataFrame(rf_grid_search.cv_results_))


Random Forest Cross-Validation Results:
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.308764      0.010338         0.032494        0.001846   
1       0.566651      0.044933         0.054582        0.015166   
2       0.821192      0.062136         0.071328        0.006273   
3       0.351188      0.037074         0.030753        0.002974   
4       0.682466      0.062876         0.050825        0.006281   
5       0.971078      0.047081         0.070907        0.011533   
6       0.428390      0.074448         0.040132        0.007650   
7       0.727753      0.031996         0.060985        0.008125   
8       1.038073      0.049785         0.084109        0.004171   

  param_max_depth param_n_estimators                                  params  \
0               5                 50    {'max_depth': 5, 'n_estimators': 50}   
1               5                100   {'max_depth': 5, 'n_estimators': 100}   
2               5                150   {'max_dep

###### Выбор лучших параметров

In [31]:
best_dt_params = dt_grid_search.best_params_
best_rf_params = rf_grid_search.best_params_
print("\nBest Decision Tree Parameters:", best_dt_params)
print("Best Random Forest Parameters:", best_rf_params)


Best Decision Tree Parameters: {'max_depth': 3}
Best Random Forest Parameters: {'max_depth': 5, 'n_estimators': 100}


### Обучение моделей с лучшими параметрами

In [32]:
best_dt_classifier = DecisionTreeClassifier(**best_dt_params)
best_rf_classifier = RandomForestClassifier(**best_rf_params)

best_dt_classifier.fit(X_train, Y_train)
best_rf_classifier.fit(X_train, Y_train)

RandomForestClassifier(max_depth=5)

###### Оценка точности на тестовом наборе

In [36]:
Y_pred_dt = best_dt_classifier.predict(X_test)
accuracy_dt = accuracy_score(Y_test, Y_pred_dt)
print("\nDecision Tree Test Accuracy:", accuracy_dt)


Decision Tree Test Accuracy: 0.9995454545454545


In [38]:
Y_pred_rf = best_rf_classifier.predict(X_test)
accuracy_rf = accuracy_score(Y_test, Y_pred_rf)
print("Random Forest Test Accuracy:", accuracy_rf)

Random Forest Test Accuracy: 0.9995454545454545


###### Создание и обучение решающего дерева

In [14]:
decision_tree = DecisionTreeClassifier(max_depth=5)  # Выбор максимальной глубины дерева
decision_tree.fit(X_train, Y_train)

DecisionTreeClassifier(max_depth=5)

###### Оценка точности на тестовом наборе

In [17]:
Y_pred_dt = decision_tree.predict(X_test)
accuracy_dt = accuracy_score(Y_test, Y_pred_dt)
print("Decision Tree Accuracy:", accuracy_dt)

Decision Tree Accuracy: 0.9986363636363637


###### Создание и обучение случайного леса

In [19]:
random_forest = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)  # Выбор количества деревьев и максимальной глубины
random_forest.fit(X_train, Y_train)

RandomForestClassifier(max_depth=10, random_state=42)

###### Оценка точности на тестовом наборе

In [20]:
Y_pred_rf = random_forest.predict(X_test)
accuracy_rf = accuracy_score(Y_test, Y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)

Random Forest Accuracy: 0.9995454545454545


###### Получение предсказаний на тестовом наборе для решающего дерева. Вывод classification_report для решающего дерева

In [41]:
Y_pred_dt = best_dt_classifier.predict(X_test)

print("Classification Report for Decision Tree:")
print(classification_report(Y_test, Y_pred_dt))

Classification Report for Decision Tree:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00       650
         Yes       1.00      1.00      1.00      1550

    accuracy                           1.00      2200
   macro avg       1.00      1.00      1.00      2200
weighted avg       1.00      1.00      1.00      2200



###### Получение предсказаний на тестовом наборе для случайного леса. Вывод classification_report для случайного леса

In [42]:
Y_pred_rf = best_rf_classifier.predict(X_test)

print("\nClassification Report for Random Forest:")
print(classification_report(Y_test, Y_pred_rf))


Classification Report for Random Forest:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00       650
         Yes       1.00      1.00      1.00      1550

    accuracy                           1.00      2200
   macro avg       1.00      1.00      1.00      2200
weighted avg       1.00      1.00      1.00      2200

